# Final Exam - Open

## FINM 37500 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# Instructions

## Please note the following:

Points
* You have `100` minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo - you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

The exam uses the following file,
`data/fiderivs_2025-03-10.xlsx`

- Section 1 uses the following sheets:
    * `rate curves`
    * quarterly spaced and quarterly compounded rates

- Section 2 uses the following sheets:
    * `rate tree`
    * continuously-compounded rate tree

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 75     |
| 2       | 25     |

Numbered problems are worth 5pts unless specified otherwise.

***

## Submitting your Exam

Submission
* You will upload your solution to the `Exam - Open` assignment on Canvas. 
* Submit a compressed, "zipped", folder containing all code according to the file structure below.
* Name your submitted, zipped, folder `exam-open-LASTNAME-FIRSTNAME.zip`.
* Be sure to **submit** on Canvas, not just **save** on Canvas.

Your submission should **include all code and data used in your analysis** in the following folder structure.
* We strongly prefer all submissions are structred this way, and it will improve grading accuracy for partial credit. 
* Still, if you're struggling to get this working in time, no worries; just structure as comfortable and submit **everything used** for your submission.

__Exam Submission Structure:__

```plaintext
exam-open-LASTNAME-FIRSTNAME.zip/
│── exam-open.ipynb
│── data/
│   ├── example_data.csv
│── modules/
│   ├── my_functions.py

### Validating your folder structure

The next cell tests that you have this folder structure implemented.

In [4]:
from pathlib import Path
import sys
import pandas as pd

# Get the directory of the notebook (assumes Jupyter Notebook is always used)
BASE_DIR = Path().resolve()

# Define paths for data and modules
DATA_DIR = BASE_DIR / "data"
MODULES_DIR = BASE_DIR / "modules"

# Check if expected directories exist
if not DATA_DIR.exists():
    print(f"Warning: Data directory '{DATA_DIR}' not found. Check your file structure.")

if not MODULES_DIR.exists():
    print(f"Warning: Modules directory '{MODULES_DIR}' not found. Check your file structure.")

# Ensure Python can import from the modules directory
if str(MODULES_DIR) not in sys.path:
    sys.path.append(str(MODULES_DIR))

# Load exam data
EXAMPLE_DATA_PATH = DATA_DIR / "fiderivs_2025-03-10.xlsx"

if EXAMPLE_DATA_PATH.exists():
    example_data = pd.read_excel(EXAMPLE_DATA_PATH)
else:
    print(f"Warning: '{EXAMPLE_DATA_PATH.name}' not found. Ensure it's in the correct directory.")


***

In [104]:
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np
from math import log, sqrt
from scipy.stats import norm

from ficcvol import *

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 1.

Consider the following rate data.

In [87]:
DATE = '2025-03-10'
FILEIN = f'./data/fiderivs_{DATE}.xlsx'

sheet_curves = f'rate curves'

curves = pd.read_excel(FILEIN, sheet_name=sheet_curves).set_index('tenor')
curves

,swap rates,spot rates,discounts,forwards,flat vols
tenor,,,,,
0.25,0.042192,0.042192,0.989562,NaN,NaN
0.50,0.040930,0.040923,0.979848,0.039655,0.146300
0.75,0.039760,0.039744,0.970775,0.037387,0.168563
1.00,0.038833,0.038808,0.962115,0.036001,0.190826
1.25,0.037868,0.037830,0.954026,0.033918,0.221870
1.50,0.037225,0.037179,0.946002,0.033928,0.244993
1.75,0.036817,0.036767,0.937960,0.034293,0.261375
2.00,0.036576,0.036524,0.929864,0.034828,0.272194
2.25,0.036284,0.036227,0.922060,0.033853,0.278629


### 1.1.

Price a fixed-rate bond with the following specifications...
* T = `5`
* coupons are paid `quarterly`.
* coupon rate is `3.6%` (quarterly).

Note: 
* Use the discount curves provided.
* Use the usual bond-pricing formula.
* The clean and dirty price are the same, as we assume the bond has just been issued.

In [88]:
T=5
COUPN_FREQ = 4
COUPN_RATE=0.0036

In [100]:
def price_bond_from_discounts(df, annual_coupon_rate = 0.036, face_value = 100.0) -> float:
    coupon_amount = annual_coupon_rate * face_value / 4.0  
    payment_tenors = [0.25 * i for i in range(1, 21)]  
    coupon_pv = 0.0
    for t in payment_tenors:
        df_t = df.loc[t, 'discounts']
        coupon_pv += coupon_amount * df_t
    df_5y = df.loc[5.0, 'discounts']
    principal_pv = face_value * df_5y
    bond_price = coupon_pv + principal_pv
    return bond_price

In [101]:
bond_price = price_bond_from_discounts(df=curves, annual_coupon_rate = 0.036, face_value = 100.0)
print(f"Bond price: {bond_price}")

Bond price: 99.94427981020664


### 1.2.

As usual, the provided cap/floor quotes correspond to caps/floors...
* notional of `$100`
* struck at-the-money, `ATM`.
* with expiration `T`.
* with `quarterly` caplets / floorlets.
* depending on the reference rate, in arrears.

Note that, as usual,
* We assume that the reference rate is compounded the same as the interest rates in the provided data. Thus, no adjustment is needed to the compounding.

Calculate and report the forward volatilities stripped from the caps.

In [102]:
capcurves = flat_to_forward_vol(curves)
forward_vols = capcurves.loc[capcurves.index[1:], 'fwd vols']
print("Forward Volatility")
display(forward_vols.to_frame())

Forward Volatility


,fwd vols
tenor,
0.50,0.146300
0.75,0.187028
1.00,0.223398
1.25,0.290121
1.50,0.306635
1.75,0.314793
2.00,0.314602
2.25,0.307474
2.50,0.298223


### 1.3. (15pts)

Price caps and floors continuing with the assumptions above. Except calculate the prices for a range of strikes...

* cap struck `ATM`.
* floor struck `ATM`.
* cap struck `+150bps`.
* floor struck `-150bps`.

Report the 
* three strikes, ATM adjusted by (-150, 0, +150).
* the dollar value of the four instruments.

Note that 
* you do not need the forward vols calculated in `1.2` to price these caps/floors. The flat vols are sufficient.

In [106]:
def blacks_formula(T,vol,strike, fwd, discount=1.0, isCall=True):
    if T <= 0 or vol <= 0 or fwd <= 0 or strike <= 0:
        return 0.0
    sigT = vol * sqrt(T)
    d1 = (log(fwd/strike) / sigT) + 0.5 * sigT
    d2 = d1 - sigT
    if isCall:
        return discount * (fwd * norm.cdf(d1) - strike * norm.cdf(d2))
    else:
        return discount * (strike * norm.cdf(-d2) - fwd * norm.cdf(-d1))


def price_cap_or_floor(strike, isCap=True, notional=100.0, freq=4, maturity=5.0):
    total_value = 0.0
    dt = 1.0 / freq
    schedule = np.arange(dt, maturity + dt, dt)
    
    for T_end in schedule:
        T_start = T_end - dt  
        if T_start <= 0:
            continue
        vol   = curves.loc[T_end, 'flat vols']
        fwd   = curves.loc[T_end, 'forwards']
        df    = curves.loc[T_end, 'discounts']
        
        accrual = dt
        isCall = isCap
        piece_value = notional * accrual * blacks_formula(
            T = T_start,
            vol = vol,
            strike = strike,
            fwd = fwd,
            discount = df,
            isCall = isCall
        )
        total_value += piece_value
    return total_value

In [112]:
atm_5y = curves.loc[5.00, 'forwards']
strike_minus_150 = atm_5y - 0.0150
strike_atm       = atm_5y
strike_plus_150  = atm_5y + 0.0150

print("ATM forward rate (5Y) =", f"{atm_5y*100:0.4f} %")
print("Strike (ATM - 150bps) =", f"{(strike_minus_150)*100:0.4f} %")
print("Strike (ATM)          =", f"{(strike_atm)*100:0.4f} %")
print("Strike (ATM + 150bps) =", f"{(strike_plus_150)*100:0.4f} %")

# i) CAP at ATM
cap_price_atm = price_cap_or_floor(strike=strike_atm, isCap=True, maturity=5.0)
# ii) FLOOR at ATM
floor_price_atm = price_cap_or_floor(strike=strike_atm, isCap=False, maturity=5.0)
# iii) CAP at (ATM + 150bps)
cap_price_plus_150 = price_cap_or_floor(strike=strike_plus_150, isCap=True, maturity=5.0)
# iv) FLOOR at (ATM - 150bps)
floor_price_minus_150 = price_cap_or_floor(strike=strike_minus_150, isCap=False, maturity=5.0)

print("\n")
print("CAP (ATM) price:          $", f"{cap_price_atm:,.6f}")
print("FLOOR (ATM) price:        $", f"{floor_price_atm:,.6f}")
print("CAP  (+150bps) price:     $", f"{cap_price_plus_150:,.6f}")
print("FLOOR (-150bps) price:    $", f"{floor_price_minus_150:,.6f}")

ATM forward rate (5Y) = 3.7200 %
Strike (ATM - 150bps) = 2.2200 %
Strike (ATM)          = 3.7200 %
Strike (ATM + 150bps) = 5.2200 %


CAP (ATM) price:          $ 2.201872
FLOOR (ATM) price:        $ 2.815914
CAP  (+150bps) price:     $ 0.840136
FLOOR (-150bps) price:    $ 0.385203


### 1.4.

Price a portfolio comprised of the positions in the table. (The positions are listed as "contracts" where each contract has face or notional of $100.)

In [114]:
atm_5y=curves.loc[5.00,'forwards']
strike_minus_150=atm_5y-0.0150
strike_atm=atm_5y
strike_plus_150=atm_5y+0.0150
bond_price=100*curves.loc[5.00,'discounts']
cap_atm=price_cap_or_floor(strike_atm,True,100,4,5.0)
floor_atm=price_cap_or_floor(strike_atm,False,100,4,5.0)
cap_otm=price_cap_or_floor(strike_plus_150,True,100,4,5.0)
floor_otm=price_cap_or_floor(strike_minus_150,False,100,4,5.0)

values=pd.DataFrame(
    data=[bond_price,cap_atm,floor_atm,cap_otm,floor_otm],
    columns=['price'],
    index=['bond','cap ATM','floor ATM','cap OTM','floor OTM']
)

In [115]:
contracts = pd.DataFrame(data=[1,1,-1,-1,1],columns = ['contracts'],index=values.index)
contracts.style.format('{:.0f}',na_rep='')

,contracts
bond,1
cap ATM,1
floor ATM,-1
cap OTM,-1
floor OTM,1


In [118]:
portfolio=values.join(contracts)
portfolio['value']=portfolio['price']*portfolio['contracts']
print(portfolio)
print(portfolio['value'].sum())

               price  contracts      value
bond       83.552697          1  83.552697
cap ATM     2.201872          1   2.201872
floor ATM   2.815914         -1  -2.815914
cap OTM     0.840136         -1  -0.840136
floor OTM   0.385203          1   0.385203
82.48372202019225


### 1.5. (10pts)

Suppose all spot rates are shocked by 1bp.

Report the new
* discount factors
* forward rates
* swap rates

(The latter two should still be quarterly compounded.)

Note that...
* We are not revising flat or forward vols.

In [35]:
def shift_discount_factors(discount_df, shift):
    tenors = discount_df.index.astype(float)
    new_discounts = discount_df["discounts"].values * np.exp(-shift * tenors)
    return pd.DataFrame(new_discounts.values, index=discount_df.index, columns=["discounts"])

shifted_df = shift_discount_factors(curves[["discounts"]], 0.0001)
shifted_df

,discounts
tenor,
0.25,0.989537
0.50,0.979799
0.75,0.970702
1.00,0.962019
1.25,0.953906
1.50,0.945860
1.75,0.937796
2.00,0.929678
2.25,0.921853


### 1.6. (10pts)

Calculate the numerical duration.
* increase all the spot rates by 1bp. 
* use the revised rates from `1.5.` corresponding to this shock.
* re-price the portfolio.

Report the 
* esitmated duration for each component of the portfolio.
* duration of the total portfolio.

### 1.7. (8pts)

The market is quoting this portfolio at a price of `100.00.`

Compute and report the **option-adjusted spread (OAS)** of this portfolio.
* Continue to model the changing rate as moving the spot rate in a parallel fashion, changing the other curves, but leaving the flat vols unchanged.

### 1.8. (5pts)

We know that the vol varies with the strike, yet we used the same implied vol for the ATM and OTM caps/floors. We now consider how much the OTM flat volatility would differ from the ATM flat volatility.

A SABR curve has been fit to OTM cap/floor flat vols, and the parameters are below.

In [36]:
sabr_parameters = pd.DataFrame(
    {'beta': 0.2500,
    'alpha': 0.0214,
    'nu': 0.6000,
    'rho': -0.2000
    },
    index=['parameter']).T

sabr_parameters.style.format('{:.4f}').set_caption('SABR (full)')

,parameter
beta,0.2500
alpha,0.0214
nu,0.6000
rho,-0.2000


Use the SABR curve to report the implied flat vols for the OTM cap and floor.
* The SABR curve requires a forward rate. Input the $T$-time forward rate provided in the data file.
* There are some nuances around using SABR for caps/floors which we are skipping. Just proceed with the instruction above.

In [40]:
STRIKE = .03
doSLIM = True

beta,alpha,nu,rho = sabr_parameters.iloc[:,0]

volOTM =  sabr(beta,nu,rho,alpha,F,STRIKE,Topt)

-0.2

### 1.9. (5pts)

No matter what you calculated in `1.8.`, proceed with implied OTM flat vols as seen in the table below

Use these to recalculate the value of the portfolio. That is, revise your answer to `1.4.`

In [24]:
flat_vols_otm = pd.DataFrame([.50,.25],index=STRIKES_OFFSET[[0,2]],columns=['flat vols'])
flat_vols_otm.index.name = 'OTM spread (bps)'
flat_vols_otm.style.format('{:.1%}')

,flat vols
OTM spread (bps),
-150,50.0%
150,25.0%


### 1.10. (7pts)

Make a plot of the portfolio value for a range of parallel interest rate shocks.

Do you see any convexity? Conceptually, (without being tied to your specific numerical answers) do you think there should be any convexity in the relationship?

***

# 2.

Consider the following interest-rate tree which fits the data from Section 1.

Rates are continuously compounded.

In [41]:
sheet_tree = 'rate tree'

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

ratetree.style.format('{:.1%}',na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,4.2%,4.2%,4.3%,4.7%,5.2%,6.6%,8.0%,9.6%,10.8%,12.5%,14.1%,15.9%,17.5%,20.1%,24.1%,27.8%,30.6%,33.4%,37.3%,41.3%
1,,3.7%,3.7%,3.9%,4.1%,5.0%,5.9%,7.0%,7.9%,9.2%,10.5%,11.9%,13.2%,15.1%,18.1%,20.8%,23.0%,25.3%,28.4%,31.6%
2,,,3.2%,3.2%,3.3%,3.7%,4.3%,5.1%,5.8%,6.8%,7.8%,8.9%,9.9%,11.3%,13.5%,15.6%,17.3%,19.1%,21.7%,24.2%
3,,,,2.7%,2.6%,2.8%,3.2%,3.7%,4.2%,5.0%,5.8%,6.7%,7.4%,8.5%,10.2%,11.7%,13.0%,14.5%,16.5%,18.5%
4,,,,,2.1%,2.1%,2.3%,2.7%,3.1%,3.7%,4.3%,5.0%,5.6%,6.4%,7.6%,8.8%,9.8%,11.0%,12.6%,14.1%
5,,,,,,1.6%,1.7%,2.0%,2.2%,2.7%,3.2%,3.7%,4.2%,4.8%,5.7%,6.6%,7.4%,8.3%,9.6%,10.8%
6,,,,,,,1.3%,1.5%,1.6%,2.0%,2.4%,2.8%,3.2%,3.6%,4.3%,4.9%,5.5%,6.3%,7.3%,8.3%
7,,,,,,,,1.1%,1.2%,1.5%,1.7%,2.1%,2.4%,2.7%,3.2%,3.7%,4.2%,4.8%,5.6%,6.3%
8,,,,,,,,,0.9%,1.1%,1.3%,1.6%,1.8%,2.0%,2.4%,2.8%,3.1%,3.6%,4.2%,4.8%


### 2.1.

Use the binomial tree to price the vanilla bond from Section 1. (Not the whole portfolio.)

Recall that the bond has `quarterly` coupons.

Report the cashflow tree of this vanilla bond.

In [53]:
def bond_cashflow_tree(states, cols, face, coupon_annual, maturity):
    cf = pd.DataFrame(0, index=states, columns=cols)
    c = face * (coupon_annual / 4)
    tm = np.array(cf.columns.tolist())
    if maturity not in tm:
        tm = np.append(tm, maturity)
    tm = np.sort(tm)
    cf = cf.reindex(columns=tm, fill_value=0)
    pt = np.arange(0.25, maturity + 0.001, 0.25)
    for p in pt:
        if p < maturity:
            cf[p] = c
        else:
            cf[p] = face + c
    return cf

In [54]:
face_value=100
coupon_rate=0.036
maturity=5
cf_tree=bond_cashflow_tree(
    states=ratetree.index,
    cols=ratetree.columns,
    face=face_value,
    coupon_annual=coupon_rate,
    maturity=maturity
)
cf_tree.style.format(na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
1,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
2,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
3,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
4,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
5,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
6,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
7,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000
8,0,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,0.900000,100.900000


### 2.2. (10pts)

Report the tree of bond values.
* Note that there is no distinction betwen clean and dirty values, as the bond pays a coupon quarterly, so every node is immediately after a coupon, at which point clean and dirty are the same.

In [55]:
def accrued_interest(t, face, coupon_annual):
    lc = 0.25 * np.floor(t / 0.25)
    f = (t - lc) / 0.25
    return face * (coupon_annual / 4) * f

def bond_value_tree(ratetree, cf_tree, face, coupon_annual):
    tm = np.sort(cf_tree.columns)
    vd = pd.DataFrame(0, index=cf_tree.index, columns=tm)
    vd.loc[:, tm[-1]] = cf_tree.loc[:, tm[-1]]
    for i in range(len(tm) - 2, -1, -1):
        dt = tm[i+1] - tm[i]
        for s in vd.index:
            sd = s + 1 if s + 1 in vd.index else s
            ev = 0.5 * vd.loc[s, tm[i+1]] + 0.5 * vd.loc[sd, tm[i+1]]
            vd.loc[s, tm[i]] = cf_tree.loc[s, tm[i]] + np.exp(-ratetree.loc[s, tm[i]] * dt) * ev
    vc = vd.copy()
    for x in tm:
        vc[x] = vc[x] - accrued_interest(x, face, coupon_annual)
    return vd, vc

In [56]:
val_dirty,val_clean=bond_value_tree(ratetree,cf_tree,face_value,coupon_rate)
val_dirty.style.format(na_rep='').format_index('{:.2f}',axis=1)

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,99.944280,98.962255,96.892738,94.612703,92.202230,89.673237,87.295730,85.061766,83.042095,81.160321,79.532735,78.153236,77.072076,76.293508,76.060287,76.768676,78.529670,81.400801,85.628486,91.900927,100.900000
1,,103.034706,101.318890,99.440485,97.431030,95.299297,93.219207,91.224555,89.376551,87.627352,86.084836,84.755835,83.686942,82.875894,82.506679,82.893151,84.115828,86.218477,89.391917,94.139360,100.900000
2,,,104.827105,103.264589,101.573593,99.762620,97.936126,96.154854,94.470199,92.854247,91.402834,90.130465,89.078919,88.241876,87.755279,87.863578,88.626062,90.079857,92.376699,95.888100,100.900000
3,,,,106.258076,104.811644,103.250964,101.632843,100.033481,98.495159,97.003268,95.640248,94.424373,93.392156,92.535140,91.951635,91.829631,92.213494,93.136778,94.722461,97.247405,100.900000
4,,,,,107.317272,105.946836,104.495515,103.046481,101.634037,100.252061,98.969758,97.806501,96.793570,95.921743,95.260777,94.953855,95.034666,95.534179,96.553232,98.299934,100.900000
5,,,,,,108.012803,106.692525,105.364998,104.057789,102.769911,101.558375,100.441972,99.447085,98.564623,97.843207,97.391098,97.234178,97.400929,97.974505,99.112523,100.900000
6,,,,,,,108.367414,107.136539,105.915508,104.706227,103.554972,102.479005,101.500304,100.610432,99.842688,99.278466,98.937825,98.846583,99.073411,99.738455,100.900000
7,,,,,,,,108.483038,107.331510,106.186694,105.085718,104.043864,103.079258,102.184386,101.381624,100.731905,100.250862,99.961497,99.920443,100.219777,100.900000
8,,,,,,,,,108.406375,107.313716,106.254017,105.240456,104.287861,103.389735,102.560789,101.846486,101.259053,100.818630,100.571792,100.589410,100.900000


### 2.3. (10pts)

Report the cashflow tree of a **structured note** defined below.

* Same maturity as the bond in `2.1`.
* Pays quarterly coupons.

But the coupon is more complicated...
* Coupon is the floating rate (in the tree)
* Paid one quarter later. (So set at $t$ and paid out at $t+.25$)
* Coupon cannot go below `2%` reference rate.
* Coupon cannot go above `5%` reference rate.

Note that unlike a vanilla bond, the cashflow depends on the node of the tree, and it determines the cashflow received one step later.

Thus the cashflow tree should show the cashflow **determined** at that node, even though it is paid out one period later. Given this, you should report the determined cashflow, discounted back one period by the continuously-compounded rate at that same node. So this discounted-determined cashflow is solelly a function of the rate at the node.

#### Careful
You are not being asked to report the **valuation** tree of the structured note--just the cashflow tree.

In [85]:
def structured_note_cashflow_tree(ratetree, face, floor_rate, cap_rate, maturity):
    times = sorted(ratetree.columns)
    cf_tree = pd.DataFrame(0.0, index=ratetree.index, columns=times)

    for s in ratetree.index:
        for t in times:
            rate = ratetree.loc[s, t]
            if int(float(t)) + 0.25 <= maturity:
                floored  = max(rate, floor_rate)
                floored_cap = min(floored, cap_rate)
                raw_coupon = face * floored_cap * 0.25
                discounted_coupon = raw_coupon * np.exp(-rate * 0.25)
                cf_tree.loc[s, t] = discounted_coupon
            
            if np.isclose(int(float(t)), maturity):
                cf_tree.loc[s, t] += face

    return cf_tree

In [86]:
structured_note_cashflow_tree(ratetree, face=100, floor_rate=0.02, cap_rate=0.05, maturity=5).style.format(na_rep='')

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,1.038313,1.047411,1.059841,1.157471,1.233997,1.229444,1.225219,1.220311,1.216648,1.211531,1.206697,1.201383,1.196399,1.188670,1.176867,1.166166,1.157827,1.149877,1.138577,1.127365
1,,0.906161,0.916931,0.961950,1.019969,1.225260,1.231715,1.228259,1.225562,1.221597,1.217717,1.213432,1.209502,1.203673,1.194780,1.186643,1.180043,1.173449,1.164199,1.155096
2,,,0.793134,0.799184,0.817453,0.919575,1.072760,1.234093,1.232111,1.229052,1.225961,1.222529,1.219443,1.215060,1.208383,1.202220,1.197026,1.191612,1.184100,1.176760
3,,,,0.663771,0.654876,0.689611,0.791729,0.926173,1.041373,1.227361,1.232115,1.229382,1.226967,1.223679,1.218679,1.214026,1.209957,1.205548,1.199485,1.193600
4,,,,,0.524459,0.516851,0.583878,0.677760,0.762447,0.905454,1.058116,1.229396,1.232652,1.230189,1.226452,1.222947,1.219771,1.216204,1.211337,1.206639
5,,,,,,0.498060,0.497844,0.497516,0.557802,0.667395,0.787434,0.923017,1.039275,1.184984,1.232311,1.229675,1.227203,1.224333,1.220443,1.216706
6,,,,,,,0.498412,0.498185,0.497957,0.497536,0.585580,0.692445,0.783022,0.892209,1.056779,1.213347,1.232820,1.230523,1.227424,1.224461
7,,,,,,,,0.498675,0.498507,0.498187,0.497819,0.519164,0.589571,0.671270,0.794106,0.911927,1.029787,1.174703,1.232768,1.230423
8,,,,,,,,,0.498910,0.498666,0.498381,0.498051,0.497777,0.504759,0.596323,0.684856,0.776280,0.891762,1.046166,1.192565


***